In [ ]:
import json
import os
import re
from collections import defaultdict

import fitz
from footnote_extractor import FootnoteExtractor
from parser_utils import (calculate_content_box, is_inside_bbox, join_jsons,
                          print_n_pages)
from table_extractor import TableExtractor
from term_extractors import BlockExtractor, RegexBlockExtractor

In [ ]:
# Muudetavad parameetrid

fdir = 'C:/Users/sandra.eiche/Documents/kood/parser_comparison/kva_data/'
fname = 'jp3_01.pdf' #os.listdir(fdir)[3]

output_fdir = 'C:\\Users\\sandra.eiche\\OneDrive - Eesti Keele Instituut\\Documents\\KVA\\kva_parsed_jsons_pymupdf'
output_fname = fname.rsplit('.', 1)[0] + '.json'

# Tabelite eraldamiseks
table_extraction_strategy = 'lines_strict' # Strateegiad kirjeldatud: https://pymupdf.readthedocs.io/en/latest/page.html#Page.find_tables, saab valida failipõhiselt sobiva strateegia.
horizontal_sorting = True # Määrata False, kui tegemist on veergudepõhise dokumendiga.

# Päiste ja jaluste eemaldamiseks
header_height = 60
footer_height = 60

# Viidete eraldamiseks
footnote_regex =  r'^\d+\s+(.*)'
footnote_group = 1

# Mõistete eraldamiseks
block_based_extraction = {
    'pages' : [(152, 157)]}

custom_regex_extraction = {
        'pages': [],
        'regular_expression': re.compile(r"(\n\s*\n|^)((.|\n)*?)(?=\n\s*\n|$)", flags=re.X), 
        #re.compile(r'((?<=  \n)|^)(.*?)(?=  \n|$)', flags=re.S)
        #(?<=^|\n)([A-Z]+.*?)(?=\n[A-Z]{2,}|$) - 
        'regex_group': 1,
        'use_blocks': True
}

In [ ]:
from parser_utils import calculate_header_and_footer_box
import matplotlib.pyplot as plt
from PIL import Image, ImageTk

document_path = os.path.join(fdir, fname)

with fitz.open(document_path) as doc:
    document_page_count = doc.page_count
    for page_no in range(0, document_page_count, round(document_page_count/6)):
        page = doc[page_no]
        header_box, footer_box = calculate_header_and_footer_box(page, header_height=header_height, footer_height=footer_height)
        shape = page.new_shape()
        shape.draw_rect(fitz.Rect(header_box))
        shape.draw_rect(fitz.Rect(footer_box))
        shape.finish(color=[0,0,1]) # Blue border, yellow fill
        shape.commit()
        pix = page.get_pixmap()


        # Convert the pixmap to a PIL image
        image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # Display the image using matplotlib
        print(page_no)
        plt.imshow(image)
        plt.axis('off') # Hide the axis
        plt.show()

In [ ]:
from pprint import pprint

with fitz.open(os.path.join(fdir, fname)) as doc:
    print(fname)
    pprint(doc.metadata)

In [ ]:
metadata = {
    'filename' : fname,
    'publication' : '',
    'publication_year' : 2017,
    'title': 'JP 3-01, Countering Air and Missile Threats',
    'author': '',
    'languages': ['en'],
    'field_keywords' : [],
    'header_height': header_height,
    'footer_height': footer_height,
    'table_extraction_strategy': table_extraction_strategy,
    'horizontal_sorting': horizontal_sorting,
    'footnote_regex': footnote_regex,
    'footnote_group': footnote_group,
    'custom_regex': custom_regex_extraction
}

---
10 lk või bloki kuvamine näitena. Saab välja võtta päiste ja jaluste koordinaadid ning need muudetavatesse parameetritesse kopeerida, et saaks neid lõike ignoreerida.
Koordinaadid on blocki 4 esimest elementi.

In [ ]:
print_n_pages(fdir, fname, display='blocks', horizontal_sorting=horizontal_sorting)

In [ ]:
print_n_pages(fdir, fname, display='text', horizontal_sorting=horizontal_sorting)

---
### Ekstraheerijate initsialiseerimine

In [ ]:
table_extractor = TableExtractor(table_extraction_strategy=table_extraction_strategy)
block_extractor = BlockExtractor()
regex_extractor = RegexBlockExtractor(regex_pattern=custom_regex_extraction['regular_expression'], 
                                      regex_group=custom_regex_extraction['regex_group'],
                                      consider_block_borders=custom_regex_extraction['use_blocks'])

footnote_extractor = FootnoteExtractor(footnote_pattern=footnote_regex, footnote_group=footnote_group)

In [ ]:
document_path = os.path.join(fdir, fname)

with fitz.open(document_path) as doc:
    table_data = table_extractor.extract_tables_from_doc(doc, header_height=header_height, footer_height=footer_height)
    block_data = block_extractor.extract_text_by_page(doc, block_based_extraction['pages'], header_height=header_height, footer_height=footer_height)
    regex_data = regex_extractor.extract_text_by_page(doc, custom_regex_extraction['pages'], header_height=header_height, footer_height=footer_height)
    footnote_data = footnote_extractor.extract_footnotes_from_doc(doc, header_height=header_height, footer_height=footer_height)
    
    term_data = join_jsons([block_data, regex_data])

In [ ]:
def is_term_page(page_no, term_data):
    """Check if the given page number is a term page."""
    for term_page in json.loads(term_data):
        if term_page['page_number'] == page_no:
            return True
    return False

In [ ]:
document_path = os.path.join(fdir, fname)

with fitz.open(document_path) as doc:
    non_table_blocks = defaultdict(list)

    content_text_data = []

    for page_no, page in enumerate(doc, 1):
        page_blocks = []

        if is_term_page(page_no, term_data):
                continue
        
        # Teksti eraldamine päist/jalust välja jättes
        text_blocks = page.get_text('blocks', sort=horizontal_sorting, clip=calculate_content_box(page, header_height, footer_height))


        for text_block in text_blocks:
            block_is_table_data = False
            block_is_footnote_data = False

            # Tabelite väljajätt
            for table_page in json.loads(table_data):
                if table_page['page_number'] == page_no:
                    for table_box in table_page['bboxes']:
                        if is_inside_bbox(text_block, table_box):
                                block_is_table_data = True
                                break
                        if block_is_table_data:
                            break

            # Joonealuste viidete väljajätt
            for footnote_page in json.loads(footnote_data):
                if footnote_page['page_number'] == page_no:
                    for footnote_box in footnote_page['bboxes']:
                        if is_inside_bbox(text_block, footnote_box):
                                block_is_footnote_data = True
                                break
                        if block_is_footnote_data:
                            break

            if not block_is_table_data and not block_is_footnote_data:
                 page_blocks.append(text_block[4])

        # Koguteksti salvestamine json-ina
        full_text_page_json = [{
            "page_number": page.number + 1,  # Page numbers are zero-based in PyMuPDF
            "text": '\n'.join(page_blocks)
            }]
        
        content_text_data.extend(full_text_page_json)

In [ ]:
content_dict = {
        'table_data': json.loads(table_data),
        'term_data': json.loads(term_data),
        'footnote_data': json.loads(footnote_data),
        'content_text_data': content_text_data,
        'term_data': json.loads(term_data)
    }

In [ ]:
content_dict

In [ ]:
print('Saving to', os.path.join(output_fdir, output_fname))
      
with open(os.path.join(output_fdir, output_fname), 'w') as fout:
          
    content_dict = {
        'table_data': json.loads(table_data),
        'term_data': json.loads(term_data),
        'footnote_data': json.loads(footnote_data),
        'content_text_data': content_text_data,
        'term_data': json.loads(term_data)
    }
          
    metadata.update(content_dict)

    custom_regex_extraction['regular_expression'] = str(custom_regex_extraction['regular_expression'])
    fout.write(json.dumps(metadata))

    

In [ ]:
json.dumps(metadata)